In [1]:
# !pip install --upgrade langchain
#!pip install --upgrade langchain-groq
# !pip install -U langchain-community
#!pip install python-docx pypandoc
# !pip install python-docx docx2pdf



In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit
from docx import Document
import pypandoc
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , create_structured_chat_agent, AgentType
import random
from groq import Groq
import streamlit as st
api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192",temperature=0.2)

In [3]:
from cryptography.fernet import Fernet
import base64
from dotenv import load_dotenv
import os
# Load environment variables from .env
load_dotenv()
with open('C:\\Users\\pc-user1\\Desktop\\key.txt', 'r') as file:
    k = file.read().strip()[2:-1]
    #k = base64.urlsafe_b64decode(k.encode())


# Decode and decrypt the API key
encoded_encrypted_api_key = os.getenv("ENCRYPTED_API_KEY")
encrypted_api_key = base64.b64decode(encoded_encrypted_api_key)
decipher_suite = Fernet(k)


In [4]:

cc_out = """
RENOVATION QUOTE DOCUMENT
**Project Overview:**
- Project Description: Renovate 150 sq ft area in basement with premium features, modern design, and a budget of $500,000.
- Length of Time for Project: 2 months

**Cost Breakdown:**

1. **Labor Costs:**
   - Total Labor Cost: $250,000

2. **Material Costs:**
   - Total Material Cost: $49,500
   - Itemized Costs:
     + 150 sq ft hardwood flooring: $5,000
     + 100 sq ft carpeting: $2,000
     + 150 sq ft drywall: $1,000
     + 50 sq ft tinted glass: $5,000
     + 100 sq ft mirror: $3,000
     + 50 sq ft modern-style wall panels: $2,000
     + 10 overhead LED lights: $1,000
     + 5 recessed LED lights: $500
     + 2 LED strip lights: $200
     + 10 new electrical outlets: $1,000
     + 2 new light switches: $200
     + 1 new dedicated 20-amp circuit: $1,000
     + 2 new vents: $1,000
     + 1 new return air vent: $500
     + 1 new water supply line: $1,000
     + 1 new drain line: $1,000
     + 100-inch TV: $5,000
     + Large couch: $2,000
     + Pool table: $2,000
     + Foosball table: $1,000
     + Hot tub: $10,000
     + Modern-style coffee table: $1,000
     + Soundproofing material: $1,000
     + Acoustic panels: $1,000
     + Joint compound: $500
     + Primer: $500
     + Painter's tape: $100

**Total Estimated Cost:**
- Total Cost of Labor: $250,000
- Total Cost of Materials: $49,500
- **Grand Total: $299,500**

**Payment Terms:**
- Deposit Required: 50% of the Grand Total ($149,750.00)
- Payment Schedule: Monthly payments of $24,958 for 6 months
- Final Payment Due: Upon completion of the project
"""

In [11]:
import os
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx2pdf import convert
from langchain.tools import BaseTool

def add_section(doc, text, font_name='Arial', font_size=12, bold=False, italic=False, alignment='left', color=None):
    # Check if the text should be a bullet point
    if text.startswith("- ") or text.startswith("+ "):
        paragraph = doc.add_paragraph(style='ListBullet')
        text = text[2:]  # Remove the bullet point indicator
    else:
        paragraph = doc.add_paragraph()
    
    run = paragraph.add_run(text)
    run.font.name = font_name
    run.font.size = Pt(font_size)
    run.bold = bold
    run.italic = italic
    if color:
        run.font.color.rgb = RGBColor(*color)
    
    if alignment == 'center':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif alignment == 'right':
        paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    else:
        paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT


def write_file(title, sections, append=False):
    documents_path = os.path.join(os.getcwd(), 'documents')
    os.makedirs(documents_path, exist_ok=True)
    docx_path = os.path.join(documents_path, f'{title}.docx')
    pdf_path = os.path.join(documents_path, f'{title}.pdf')
    
    if append and os.path.exists(docx_path):
        doc = Document(docx_path)
    else:
        doc = Document()

    for section in sections:
        add_section(
            doc,
            section["content"],
            font_name=section.get("font_name", "Arial"),
            font_size=section.get("font_size", 12),
            bold=section.get("bold", False),
            italic=section.get("italic", False),
            alignment=section.get("alignment", "left"),
            color=section.get("color", None)
        )
    
    doc.save(docx_path)
    convert(docx_path, pdf_path)

    return f"Document written and converted to pdf. No further action needed."


class WriteFileTool(BaseTool):
    name = "write_file"
    description = "Writes content into a .docx file with specified formatting and converts it to a .pdf file."
    
    def _run(self, title, sections, append=False):
        self.process_sections_in_batches(title, sections, append)

    def process_sections_in_batches(self, title, sections, append, batch_size=1):
        for i in range(0, len(sections), batch_size):
            batch = sections[i:i+batch_size]
            result = write_file(title, batch, append=(i > 0))
        return ("STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE!")

write_file_tool = WriteFileTool()


import re

def parse_formatted_response(response):
    title_match = re.search(r'"title":\s*"([^"]+)"', response)
    title = title_match.group(1) if title_match else "Untitled Document"
    
    sections_matches = re.findall(r'\{([^}]+)\}', response)
    sections = []
    for match in sections_matches:
        section = {}
        content_match = re.search(r'"content":\s*"([^"]+)"', match)
        font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
        font_size_match = re.search(r'"font_size":\s*(\d+)', match)
        bold_match = re.search(r'"bold":\s*(true|false)', match)
        italic_match = re.search(r'"italic":\s*(true|false)', match)
        alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
        color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
        section["content"] = content_match.group(1) if content_match else ""
        section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
        section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
        section["bold"] = bold_match.group(1) == "true" if bold_match else False
        section["italic"] = italic_match.group(1) == "true" if italic_match else False
        section["alignment"] = alignment_match.group(1) if alignment_match else "left"
        if color_match:
            section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
        sections.append(section)
    
    append_match = re.search(r'"append":\s*(true|false)', response)
    append = append_match.group(1) == "true" if append_match else False
    
    return title, sections, append





chat_completion = client.chat.completions.create(
    messages=[
        {
            "role":"system",
            "content":"You are a professional formatter. BE ELABORATE."
        },
        {
            "role":"user",
            "content":"please synthesize ALL of this stuff into sentence format. Start the paragraph with '@' character. put the itemized costs in parentheses beside the item: " + cc_out,
        }
    ],
    model = "llama3-70b-8192",
)

cc_out2 = (chat_completion.choices[0].message.content)
idx = cc_out2.find('@')
cc_out2 = cc_out2[idx+1:].strip()
print('ccount2 old\n',cc_out2)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """You are a text formatting expert. 
            You are an expert also at determining what numerous text formatting parameters should be.
            
                        You must always output in a structured format as follows:
                        {
                            "title": "The Great Renovation",
                            "sections": [
                                {
                                    "content": "<specific text content>",
                                    "font_name": "<font family>",
                                    "font_size": <integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                },
                                {
                                    "content": "<another specific text content>",
                                    "font_name": "<another font family>",
                                    "font_size": <another integer specifying font size>,
                                    "bold": <true or false>,
                                    "italic": <true or false>,
                                    "alignment": "<right, left, justify or center>"
                                    "color": [<integer for red>, <integer for green>, <integer for blue>]
                                }
                                // ... more sections as needed
                            ],
                            "append": <true or false> (this indicates appending to document or not)
                        }
                        Ensure the output is well-formed and valid."""
        },
        {
            "role": "user",
            "content": "TITLE OF DOCUMENT IS 'The Great Renovation'. Include titles, headings, format in a nice readable and coherent way, NOT a big block of text.  \
                Specify the formatting for this text. Please make it a \
                    PROFESSIONALLY formatted text WITH black blue colors and times new roman font PLEASE. : " + cc_out2
        }
    ],
    model="llama3-70b-8192",
)

cc_out2 = chat_completion.choices[0].message.content

print('CCOUT2\n',cc_out2)

# title, sections, append = parse_formatted_response(cc_out2)

# write_file_tool._run(title, sections, append)


ccount2 old
 This renovation project, set to last 2 months, aims to transform a 150 sq ft basement area into a premium, modern space within a budget of $500,000, featuring labor costs of $250,000, and material costs of $49,500, broken down into: 150 sq ft hardwood flooring ($5,000), 100 sq ft carpeting ($2,000), 150 sq ft drywall ($1,000), 50 sq ft tinted glass ($5,000), 100 sq ft mirror ($3,000), 50 sq ft modern-style wall panels ($2,000), 10 overhead LED lights ($1,000), 5 recessed LED lights ($500), 2 LED strip lights ($200), 10 new electrical outlets ($1,000), 2 new light switches ($200), 1 new dedicated 20-amp circuit ($1,000), 2 new vents ($1,000), 1 new return air vent ($500), 1 new water supply line ($1,000), 1 new drain line ($1,000), a 100-inch TV ($5,000), a large couch ($2,000), a pool table ($2,000), a foosball table ($1,000), a hot tub ($10,000), a modern-style coffee table ($1,000), soundproofing material ($1,000), acoustic panels ($1,000), joint compound ($500), primer 

In [6]:
zapier = ZapierNLAWrapper(zapier_nla_api_key=decipher_suite.decrypt(encrypted_api_key).decode())
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
import json
from langchain.agents import Tool
my_tools = []
for tool in toolkit.tools:
    my_tools.append(tool)


my_tools.append(
    
    write_file_tool
)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [7]:
# for tool in my_tools:
#     print(tool.name)
#     print(tool.description)
#     print("\n\n")

In [12]:
title, sections, append = parse_formatted_response(cc_out2)

result = agent.invoke({"input": f"use write file tool --> TITLE: {title}, SECTIONS: {json.dumps(sections)}, APPEND: {append}"})
print(result)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "write_file",
  "action_input": {
    "title": "The Great Renovation",
    "sections": [{"content": "The Great Renovation", "font_name": "Times New Roman", "font_size": 24, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Project Overview", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 255]}, {"content": "This renovation project, set to last 2 months, aims to transform a 150 sq ft basement area into a premium, modern space within a budget of $500,000.", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Budget Breakdown:", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 255]}, {"content": "Labor Costs: $250,000\nMaterial Costs: $49,500", "font_name": "Times New 

100%|██████████| 1/1 [00:01<00:00,  1.79s/it]



Observation: None
Thought:Action:
```
{
  "action": "Final Answer",
  "action_input": "The file has been successfully created with the specified content and formatting."
}
```


> Finished chain.
{'input': 'use write file tool --> TITLE: The Great Renovation, SECTIONS: [{"content": "The Great Renovation", "font_name": "Times New Roman", "font_size": 24, "bold": true, "italic": false, "alignment": "center", "color": [0, 0, 0]}, {"content": "Project Overview", "font_name": "Times New Roman", "font_size": 18, "bold": true, "italic": false, "alignment": "left", "color": [0, 0, 255]}, {"content": "This renovation project, set to last 2 months, aims to transform a 150 sq ft basement area into a premium, modern space within a budget of $500,000.", "font_name": "Times New Roman", "font_size": 12, "bold": false, "italic": false, "alignment": "left", "color": [0, 0, 0]}, {"content": "Budget Breakdown:", "font_name": "Times New Roman", "font_size": 14, "bold": true, "italic": false, "alignment":